In [3]:
import os
import openai

In [2]:
!pip install openai

  Using cached openai-0.27.2-py3-none-any.whl (70 kB)
     ---------------------------------------- 0.0/319.8 kB ? eta -:--:--
     --- ----------------------------------- 30.7/319.8 kB 1.4 MB/s eta 0:00:01
     ----- ------------------------------- 51.2/319.8 kB 440.4 kB/s eta 0:00:01
     ------------ ----------------------- 112.6/319.8 kB 819.2 kB/s eta 0:00:01
     ----------------------- -------------- 194.6/319.8 kB 1.1 MB/s eta 0:00:01
     --------------------------- ---------- 235.5/319.8 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 319.8/319.8 kB 1.2 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
     ---------------------------------------- 0.0/56.1 kB ? eta -:--:--
     ---------------------------------------- 56.1/56.1 kB 2.9 MB/s eta 0:00:00


In [4]:
openai.api_key = os.getenv('OPEN_API_KEY')

In [6]:
from git import Repo

In [7]:
from pathlib import Path

In [8]:
pwd

'C:\\Users\\Lenovo\\Documents\\GitHub\\SNEHAASHISH.github.io'

In [22]:
PATH_TO_STORY_REPO = Path("C:\\Users\\Lenovo\\Documents\\GitHub\\SNEHAASHISH.github.io\\.git")

In [23]:
PATH_TO_STORY = PATH_TO_STORY_REPO.parent

In [24]:
PATH_TO_CONTENT = PATH_TO_STORY/"content"

In [25]:
PATH_TO_CONTENT

WindowsPath('C:/Users/Lenovo/Documents/GitHub/SNEHAASHISH.github.io/content')

In [26]:
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [28]:
def update_story(commit_msg='Upades story'):
    repo = Repo(PATH_TO_STORY_REPO)
    #git add .
    repo.git.add(all=True)
    #git commit -m "updates story"
    repo.index.commit(commit_msg)
    #git push
    origin = repo.remote(name='origin')
    origin.push()

In [29]:
random_text_string = "wfarfwvwfnjlvbwefcnwbjvwlfhbejedcbhrvblf"

In [30]:
with open(PATH_TO_STORY/"index.html","w") as f:
    f.write(random_text_string)

In [31]:
update_story()

In [32]:
import shutil
def create_new_story(title,content,cover_img):
    cover_img = Path(cover_img)
    
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_img,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        #WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write('<!DOCTYPE HTML> \n')
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>")
            f.write("<body>")
            f.write(f"<img src='{cover_img.name}' alt='Cover Image'> <br/>\n")
            f.write(f"<h1> {title} </h1>")
            f.write(content.replace("\n","<br/>\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Story Created")
            return path_to_new_content
    else:
        raise FileExistsError("File Already Exists, please check again ur name! ABORTING!")

In [34]:
path_to_new_content = create_new_story('Test_Title','weweccwwecwr','coconut-tree.png')

Story Created


In [35]:
# index.html --> Stories

In [36]:
from bs4 import BeautifulSoup as Soup

In [37]:
with open(PATH_TO_STORY/"index.html") as index:
    soup = Soup(index.read())

In [38]:
str(soup)

'wfarfwvwfnjlvbwefcnwbjvwlfhbejedcbhrvblf'